In [ ]:
import json
import pandas as pd
import requests
import sqlalchemy
from datetime import datetime, date, timedelta
from pytz import timezone



#FUNCTION with list of cities
def get_city_data(cities):
    API_key = 'INSERT OR IMPORT API KEY HERE'
    city_dict = {
            'city': [],
            'country_code': [],
            'population': [],
            'latitude': [],
            'longitude': [],
            'timezone': [],
            'sunrise': [],
            'sunset': []
            # 'city_id': []
            }

    for j in range(len(cities)):
        weather_req = requests.get(f"http://api.openweathermap.org/data/2.5/forecast?q={cities[j]}&appid={API_key}&units=metric")
        city_json = weather_req.json()
        city_dict['city'].append(city_json['city']['name'])
        city_dict['country_code'].append(city_json['city']['country'])
        city_dict['population'].append(city_json['city']['population'])
        city_dict['latitude'].append(city_json['city']['coord']['lat'])
        city_dict['longitude'].append(city_json['city']['coord']['lon'])
        city_dict['timezone'].append(city_json['city']['timezone'])
        city_dict['sunrise'].append(city_json['city']['sunrise'])
        city_dict['sunset'].append(city_json['city']['sunset'])
        # city_dict['city_id'].append(str(uuid.uuid4()))

        

    city_df = pd.DataFrame(city_dict)
    return city_df

def get_weather_data(cities):
    # cities = ['Bishkek', 'Florence']
    API_key = 'INSERT OR IMPORT API KEY HERE'

    city_weather_dict = {
            'city': [],
            'country_code': [],
            'forecast_time': [],
            'outlook': [],
            'temp': [],
            'feels_like': [],
            'wind_speed': [],
            'humidity': []
            }

    for j in range(len(cities)):
        weather_req = requests.get(f"http://api.openweathermap.org/data/2.5/forecast?q={cities[j]}&appid={API_key}&units=metric")
        city_weather_json = weather_req.json()

        for i in range(len(city_weather_json['list'])):
            city_weather_dict['city'].append(city_weather_json['city']['name'])
            city_weather_dict['country_code'].append(city_weather_json['city']['country'])
            city_weather_dict['forecast_time'].append(city_weather_json['list'][i]['dt_txt'])
            city_weather_dict['outlook'].append(city_weather_json['list'][i]['weather'][0]['description'])
            city_weather_dict['temp'].append(city_weather_json['list'][i]['main']['temp'])
            city_weather_dict['feels_like'].append(city_weather_json['list'][i]['main']['feels_like'])
            city_weather_dict['wind_speed'].append(city_weather_json['list'][i]['wind']['speed'])
            city_weather_dict['humidity'].append(city_weather_json['list'][i]['main']['humidity'])
        

    city_weather_df = pd.DataFrame(city_weather_dict)
    return city_weather_df
    
#Function to get the data from the API

def tomorrow_arrivals(airports):

    today = datetime.now().astimezone(timezone('Europe/Berlin')).date()
    tomorrow = (today + timedelta(days=1))
    arrivals_dict = {
        'departure_icao': [],
        'city': [],
        'actual_arrival': [],
        'scheduled_arrival': [],
        'terminal': [],
        'airline': [],
        'arrival_icao': []
        }

    for airport in airports:
        times = [["00:00","11:59"],["12:00","23:59"]]
        # loop through the airports list
        for time in times:
            # the url, querystring and headers were created in rapidapi website and copied here
            
            
            url = f"https://aerodatabox.p.rapidapi.com/flights/airports/icao/{airport}/{tomorrow}T{time[0]}/{tomorrow}T{time[1]}"
            querystring = {"direction":"Arrival","withCancelled":"true","withCodeshared":"false","withLocation":"false"}
            
            headers = {
                "X-RapidAPI-Key": "INSERT FLIGHTS_API_KEY HERE",
                "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
            }
            # make the request
            response = requests.request("GET", url, headers=headers, params=querystring)
            # get the json data
            time_json_data = response.json()['arrivals']
            
            for flight in time_json_data:
                # if flight['movement']['airport'].get("icao") in airports:
                    arrivals_dict['departure_icao'].append(flight['movement']['airport'].get("icao", None))
                    arrivals_dict['city'].append(flight['movement']['airport'].get('name', None))
                    arrivals_dict['actual_arrival'].append(flight['movement'].get('scheduledTimeUtc', None))
                    arrivals_dict['scheduled_arrival'].append(flight['movement'].get('scheduledTimeLocal', None))
                    arrivals_dict['terminal'].append(flight['movement'].get('terminal', None))
                    arrivals_dict['airline'].append(flight['airline'].get('name', None))
                    arrivals_dict['arrival_icao'].append(airport)  

    arrivals_df = pd.DataFrame(arrivals_dict)
    return arrivals_df
    
def lambda_handler(event, context):
    # TODO implement
    schema="weather_pipelines"
    host="wbs-clouding-db.cskrwsswrpjx.eu-north-1.rds.amazonaws.com"  
    user="admin"
    password="INSERT AWS PASSWORD HERE"
    port=3306
    con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'
    # cities = get_city_data()['city'].to_list()
    #retrieves/pulls "city_airport" table from AWS MySQL and creates read DataFrame here in lambda
    # city_airport_df = pd.read_sql('city_airport',con=con)
    
    #create lists from the 'city_airport' table that we created inside AWS MySQL. These lists are passed as arguments into functions to create DFs as below.
    #we create each variable list from columns in 'city_airport' table
    cities = pd.read_sql('city_airport', con=con)["city"].to_list()
    airports = pd.read_sql('city_airport', con=con)['icao'].to_list()
    
    #CREATE DFs from functions above
    city_weather_df = get_weather_data(cities)
    city_df = get_city_data(cities)
    arrivals_df = tomorrow_arrivals(airports)
    
    #CREATE TABLES FROM DFs in AWS MySQL
    city_weather_df.to_sql('weather',con=con,if_exists='append',index=False)
    arrivals_df.to_sql('arrivals',con=con,if_exists='append',index=False)
    
    #if you run lambda function with existing city_df or change cities with hard-coded values like with this list: # cities = ['Bishkek', 'Istanbul', 'Florence'] 
    # then it is better to use 'replace' instead of 'append' function because the values in cities table in MySQL might be duplicated and added on top
    city_df.to_sql('cities',con=con,if_exists='replace',index=False)
    return {
        'statusCode': 200,
        'body': json.dumps('Hello from Lambda!')
    }
